1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv.

In [14]:
import pandas 
import re

data = pandas.read_csv(r"C:\Users\USER\Desktop\ML\ML_intro\10-statement-linreg\salary-train.csv")
X = data.loc[:, "FullDescription":"ContractTime"]
y = data['SalaryNormalized']

X.head()

,FullDescription,LocationNormalized,ContractTime
0,International Sales Manager London ****k ****...,London,permanent
1,An ideal opportunity for an individual that ha...,London,permanent
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent
3,A great local marketleader is seeking a perman...,Dereham,permanent
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN


2. Проведите предобработку:
• Приведите тексты к нижнему регистру.
• Замените все, кроме букв и цифр, на пробелы — это облегчит
дальнейшее разделение текста на слова. Для такой замены в
строке text подходит следующий вызов:
2
r e . sub ( ’ [^ a−zA−Z0−9] ’ , ’ ␣ ’ , t e x t . low e r ( ) )
• Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).
• Замените пропуски в столбцах LocationNormalized и ContractTime
на специальную строку ’nan’. Код для этого был приведен выше.
• Примените DictVectorizer для получения one-hot-кодирования
признаков LocationNormalized и ContractTime.
• Объедините все полученные признаки в одну матрицу "объектыпризнаки". Обратите внимание, что матрицы для текстов и
категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack

X["FullDescription"] = X["FullDescription"].str.lower().str.replace('[^a-zA-Z0-9]', ' ', regex=True)
X['LocationNormalized'].fillna('nan', inplace=True)
X['ContractTime'].fillna('nan', inplace=True)

vectorize = TfidfVectorizer(min_df=5)
vectorize_matrix = vectorize.fit_transform(X['FullDescription'])

dict = DictVectorizer()
encoded = dict.fit_transform(X[['LocationNormalized', 'ContractTime']].to_dict('records'))

X_train = hstack([vectorize_matrix, encoded])
Y_train = data['SalaryNormalized']

            


3. Обучите гребневую регрессию с параметром alpha=1. Целевая переменная записана в столбце SalaryNormalized.

In [16]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=1)
clf.fit(X_train, Y_train)

Ridge(alpha=1)

4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv.
Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.


In [26]:
data_test = pandas.read_csv(r"C:\Users\USER\Desktop\ML\ML_intro\10-statement-linreg\salary-test-mini.csv")

X_test = data_test.loc[:, "FullDescription":"ContractTime"]
y_test = data_test['SalaryNormalized']

X_test["FullDescription"] = X_test["FullDescription"].str.lower().str.replace('[^a-zA-Z0-9]', ' ', regex=True)
X_test['LocationNormalized'].fillna('nan', inplace=True)
X_test['ContractTime'].fillna('nan', inplace=True)
encoded_test = dict.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))
vectorize_matrix_test = vectorize.transform(data_test['FullDescription'])

X_train_test = hstack([vectorize_matrix_test, encoded_test])
Y_train_test = data_test['SalaryNormalized']

predictions = clf.predict(X_train_test)
print(predictions)


[56577.50021045 37199.99984646]
